In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import resgen as rg

rgc = rg.connect(os.getenv('RGU'), os.getenv('RGP'))

In [3]:
project = rgc.find_or_create_project('Lhoumaud et al. 2019')

In [4]:
h3k36m3_high1 = project.sync_dataset(
    'ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3791nnn/GSM3791763/suppl/GSM3791763_K36me2_High1_SI.bw',
    assembly='hg38'
)

In [5]:
h3k36m3_low1 = project.sync_dataset(
    'ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3791nnn/GSM3791765/suppl/GSM3791765_K36me2_Low1_SI.bw',
    assembly='hg38'
)

In [6]:
h3k27ac_high1 = project.sync_dataset(
    'ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3791nnn/GSM3791753/suppl/GSM3791753_KMS11_H3K27ac_RPKM.bw',
    assembly='hg38'
)

In [7]:
h3k27ac_low1 = project.sync_dataset(
    'ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM3791nnn/GSM3791756/suppl/GSM3791756_TKO1_H3K27ac_RPKM.bw',
    assembly='hg38'
)

In [8]:
nsd2_high = project.sync_dataset(
    'GSE131651_NSD2_HIGH_arima.mcool',
    assembly='hg19'
)

In [9]:
nsd2_low = project.sync_dataset(
    'GSE131651_NSD2_LOW_arima.mcool',
    assembly='hg19'
)

In [11]:
chromsizes = rgc.find_datasets(
  datatype='chromsizes', assembly='hg38'
)[0]
chrominfo = rgc.get_chrominfo(chromsizes)

In [12]:
gene_annotations = rgc.find_datasets(
    datatype='gene-annotations', assembly='hg38'
)[0]

In [20]:
from higlass import Track, View
import higlass

initialXDomain = chrominfo.to_abs_range('chr10', 34820000, 38820001)

tr1 = h3k36m3_high1.hg_track()
tr2 = h3k36m3_low1.hg_track()

linear_view = View(
            [
                chromsizes.hg_track(tickPositions='ends'),
                ( h3k27ac_high1.hg_track() / h3k27ac_low1.hg_track()).change_attributes(
                    height=100, type='horizontal-divergent-bar', position='top',
                    options={
                        "valueScaling": "log"
                    }
                ),
                tr1,
                tr2,
                (tr1 / tr2).change_attributes(
                    height=100, type='horizontal-divergent-bar', position='top'),
                gene_annotations.hg_track()
            ],
            initialXDomain = initialXDomain, width=6
        )

(d,s,v) = higlass.display(
    [linear_view],
    auth_token=f"JWT {rgc.get_token()}")
d

HiGlassDisplay(auth_token='JWT eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJ1c2VyX2lkIjoyLCJ1c2VybmFtZSI6InBldGUiLC…

In [23]:
chromsizes_hg19 = rgc.find_datasets(
  datatype='chromsizes', assembly='hg19'
)[0]
chrominfo_hg19 = rgc.get_chrominfo(chromsizes_hg19)

In [29]:
from higlass.utils import hg_cmap

heatmap_options = {
    "height": 200,
    "showMousePosition": "true",
}

initialXDomain_hg19 = chrominfo_hg19.to_abs_range('chr10', 34000000, 39000000)


views = [
#     linear_view,
    View([
        chromsizes_hg19.hg_track(tickPositions='ends'),
        nsd2_high.hg_track(**heatmap_options)
    ], initialXDomain=initialXDomain_hg19, width=4, x=0, y=0),
    View([
        chromsizes_hg19.hg_track(tickPositions='ends'),
        nsd2_low.hg_track(**heatmap_options),
    ], initialXDomain=initialXDomain_hg19, width=4, x=9, y=0),
    View([
        chromsizes_hg19.hg_track(tickPositions='ends'),
        (nsd2_high.hg_track() / nsd2_low.hg_track()).change_attributes(
            **{
                **heatmap_options,
                "options": {
                    "colorRange": hg_cmap('jet', reverse=True),
                    "showMousePosition": "true"
                }
            }
        )
    ], initialXDomain=initialXDomain_hg19, x=4, width=4, y=0)
]

(d,s,v) = higlass.display(views, location_syncs=[views], zoom_syncs=[views])
d

HiGlassDisplay(viewconf={'editable': True, 'views': [{'uid': 'NVoAHsSDT3Gsu37OxF_RzA', 'tracks': {'top': [{'ty…

In [18]:
project.sync_viewconf(v, "Figure 4 (Draft)")

In [32]:
project.sync_viewconf(d.viewconf, 'Screenshot 1')